In [1]:
# Random forest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
data=pd.read_csv(file_url)

In [2]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [4]:
round(data.describe(), 2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


##### data['engine']을 숫자형으로 변형

In [5]:
data['engine'].str.split()

0       [1248, CC]
1       [1498, CC]
2       [1497, CC]
3       [1396, CC]
4       [1298, CC]
           ...    
8123    [1197, CC]
8124    [1493, CC]
8125    [1248, CC]
8126    [1396, CC]
8127    [1396, CC]
Name: engine, Length: 8128, dtype: object

In [6]:
data['engine'].str.split(expand=True) # 공백 기준으로 문자를 분할하여 별도의 변수로 출력


,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [7]:
data[['engine', 'engine_unit']] = data['engine'].str.split(expand=True)
data

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,engine_unit
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248,74 bhp,190Nm@ 2000rpm,5.0,CC
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498,103.52 bhp,250Nm@ 1500-2500rpm,5.0,CC
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,CC
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396,90 bhp,22.4 kgm at 1750-2750rpm,5.0,CC
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,CC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai i20 Magna,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197,82.85 bhp,113.7Nm@ 4000rpm,5.0,CC
8124,Hyundai Verna CRDi SX,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493,110 bhp,"24@ 1,900-2,750(kgm@ rpm)",5.0,CC
8125,Maruti Swift Dzire ZDi,2009,382000,120000,Diesel,Individual,Manual,First Owner,19.3 kmpl,1248,73.9 bhp,190Nm@ 2000rpm,5.0,CC
8126,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396,70 bhp,140Nm@ 1800-3000rpm,5.0,CC


In [8]:
data['engine'] = data['engine'].astype('float32') # 숫자형 변수로 변환

In [9]:
data['engine'].dtype

dtype('float32')

In [10]:
data['engine'].head()

0    1248.0
1    1498.0
2    1497.0
3    1396.0
4    1298.0
Name: engine, dtype: float32

In [11]:
data.drop('engine_unit', inplace=True, axis = 1)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   float32
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float32(1), float64(1), int64(3), object(8)
memory usage: 793.9+ KB


##### data['max_power']을 숫자형으로 변형

In [13]:
data[['max_power', 'max_power_unit']] = data['max_power'].str.split(expand=True)
data

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248.0,74,190Nm@ 2000rpm,5.0,bhp
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498.0,103.52,250Nm@ 1500-2500rpm,5.0,bhp
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497.0,78,"12.7@ 2,700(kgm@ rpm)",5.0,bhp
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396.0,90,22.4 kgm at 1750-2750rpm,5.0,bhp
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298.0,88.2,"11.5@ 4,500(kgm@ rpm)",5.0,bhp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai i20 Magna,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197.0,82.85,113.7Nm@ 4000rpm,5.0,bhp
8124,Hyundai Verna CRDi SX,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493.0,110,"24@ 1,900-2,750(kgm@ rpm)",5.0,bhp
8125,Maruti Swift Dzire ZDi,2009,382000,120000,Diesel,Individual,Manual,First Owner,19.3 kmpl,1248.0,73.9,190Nm@ 2000rpm,5.0,bhp
8126,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396.0,70,140Nm@ 1800-3000rpm,5.0,bhp


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            8128 non-null   object 
 1   year            8128 non-null   int64  
 2   selling_price   8128 non-null   int64  
 3   km_driven       8128 non-null   int64  
 4   fuel            8128 non-null   object 
 5   seller_type     8128 non-null   object 
 6   transmission    8128 non-null   object 
 7   owner           8128 non-null   object 
 8   mileage         7907 non-null   object 
 9   engine          7907 non-null   float32
 10  max_power       7913 non-null   object 
 11  torque          7906 non-null   object 
 12  seats           7907 non-null   float64
 13  max_power_unit  7906 non-null   object 
dtypes: float32(1), float64(1), int64(3), object(9)
memory usage: 857.4+ KB


In [15]:
data[data['max_power'] == 'bhp']

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


In [16]:
def isFloat(value): 
    try: 
        num = float(value)
        return num 
    except ValueError:
        return np.NaN

In [17]:
data['max_power'] = data['max_power'].apply(isFloat)

In [18]:
data['max_power_unit'].value_counts()

bhp    7906
Name: max_power_unit, dtype: int64

In [19]:
data.drop('max_power_unit', axis=1, inplace=True)

##### data['mileage']을 숫자형으로 변형

In [20]:
data[['mileage', 'mileage_unit']] = data['mileage'].str.split(expand=True)

In [21]:
data['mileage'].head()

0     23.4
1    21.14
2     17.7
3     23.0
4     16.1
Name: mileage, dtype: object

In [22]:
data['mileage'] = data['mileage'].astype('float32')

In [23]:
data['mileage_unit'].value_counts()

kmpl     7819
km/kg      88
Name: mileage_unit, dtype: int64

In [24]:
# kmpl: 킬로그램당 킬로미터 km/kg : 휘발유와 디젤은 리터 단위로 측정하지만, LPG나 CNG는 킬로그램 단위

In [25]:
data['fuel'].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [26]:
# 1달러당 몇킬로미터를 달릴 수 있는지 계산 
def mile(x): 
    if x['fuel'] == 'Petrol':
        return x['mileage'] / 80.43
    elif x['fuel'] == 'Diesel': 
        return x['mileage'] / 73.56
    elif x['fuel'] == 'LPG':
        return x['mileage'] / 40.85
    else:
        return x['mileage'] / 44.23


In [27]:
data['mileage'] = data.apply(mile, axis=1)

In [28]:
data.drop('mileage_unit', axis=1, inplace=True)

##### torque 변수 처리하기

In [29]:
data['torque']

0                  190Nm@ 2000rpm
1             250Nm@ 1500-2500rpm
2           12.7@ 2,700(kgm@ rpm)
3        22.4 kgm at 1750-2750rpm
4           11.5@ 4,500(kgm@ rpm)
                  ...            
8123             113.7Nm@ 4000rpm
8124    24@ 1,900-2,750(kgm@ rpm)
8125               190Nm@ 2000rpm
8126          140Nm@ 1800-3000rpm
8127          140Nm@ 1800-3000rpm
Name: torque, Length: 8128, dtype: object

In [30]:
data['torque'] = data['torque'].str.upper()

In [33]:
def torque_unit(x):
    if 'NM' in str(x):
        return 'Nm'
    elif 'KGM' in str(x):
        return 'kgm'
    
data['torque_unit'] = data['torque'].apply(torque_unit)

In [34]:
data['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [37]:
data[data['torque_unit'].isna()]['torque'].unique()

array([nan, '250@ 1250-5000RPM', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [38]:
# Nm, kgm도 없는 데이터와 NaN으로 입력된 데이터 => Nm, kgm중 하나에 속한다고 추론할 때 Nm은 보통 100, 200 이고 kgm은 10, 20eksdnldlek. 
# 숫자 크기를 고려할 때 모두 Nm에 속한다고 추론가능하다 

data['torque_unit'].fillna('Nm', inplace=True)

In [39]:
string_example = '12.7@ 2,700(KGM@ RPM)'

In [42]:
for i, x in enumerate(string_example):
    print(i, '번째 텍스트', x)

0 번째 텍스트 1
1 번째 텍스트 2
2 번째 텍스트 .
3 번째 텍스트 7
4 번째 텍스트 @
5 번째 텍스트  
6 번째 텍스트 2
7 번째 텍스트 ,
8 번째 텍스트 7
9 번째 텍스트 0
10 번째 텍스트 0
11 번째 텍스트 (
12 번째 텍스트 K
13 번째 텍스트 G
14 번째 텍스트 M
15 번째 텍스트 @
16 번째 텍스트  
17 번째 텍스트 R
18 번째 텍스트 P
19 번째 텍스트 M
20 번째 텍스트 )


In [43]:
for i, x in enumerate(string_example): 
    if x not in '0123456789.':
        cut = i
        break

In [48]:
def split_num(s): 
    s = str(s)
    for i, x in enumerate(s):
        if x not in '0123456789.':
            cut = i
            break
    return s[:cut]

In [50]:
data['torque'] = data['torque'].apply(split_num)

In [53]:
data['torque'] = data['torque'].replace('', np.NaN)
data['torque'] = data['torque'].astype('float64')

In [54]:
data['torque'].head()

0    190.0
1    250.0
2     12.7
3     22.4
4     11.5
Name: torque, dtype: float64

In [55]:
def torque_trans(x):
    if x['torque_unit'] == 'kgm':
        return x['torque'] * 9.8066
    else: 
        return x['torque']

In [56]:
data['torque'] = data.apply(torque_trans, axis = 1)

In [57]:
data.drop('torque_unit', axis=1, inplace=True)

In [58]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,0.318108,1248.0,74.00,190.00000,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,0.287384,1498.0,103.52,250.00000,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,0.220067,1497.0,78.00,124.54382,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,0.312670,1396.0,90.00,219.66784,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,0.200174,1298.0,88.20,112.77590,5.0


In [61]:
data.describe(include='all')

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
count,8128,8128.000000,8.128000e+03,8.128000e+03,8128,8128,8128,8128,7907.000000,7907.000000,7912.000000,7906.000000,7907.000000
unique,2058,NaN,NaN,NaN,4,3,2,5,NaN,NaN,NaN,NaN,NaN
top,Maruti Swift Dzire VDI,NaN,NaN,NaN,Diesel,Individual,Manual,First Owner,NaN,NaN,NaN,NaN,NaN
freq,129,NaN,NaN,NaN,4402,6766,7078,5289,NaN,NaN,NaN,NaN,NaN
mean,NaN,2013.804011,6.382718e+05,6.981951e+04,NaN,NaN,NaN,NaN,0.256477,1458.625000,91.517919,180.602551,5.416719
std,NaN,4.044249,8.062534e+05,5.655055e+04,NaN,NaN,NaN,NaN,0.062588,503.920441,35.822499,109.252110,0.959588
min,NaN,1983.000000,2.999900e+04,1.000000e+00,NaN,NaN,NaN,NaN,0.000000,624.000000,0.000000,47.071680,2.000000
25%,NaN,2011.000000,2.549990e+05,3.500000e+04,NaN,NaN,NaN,NaN,0.216966,1197.000000,68.050000,111.800000,5.000000
50%,NaN,2015.000000,4.500000e+05,6.000000e+04,NaN,NaN,NaN,NaN,0.250404,1248.000000,82.000000,170.000000,5.000000
75%,NaN,2017.000000,6.750000e+05,9.800000e+04,NaN,NaN,NaN,NaN,0.294454,1582.000000,102.000000,209.000000,5.000000


In [63]:
data['name'] = data['name'].str.split(expand=True)[0]

In [64]:
data['name'].unique()

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

In [65]:
data['name'] = data['name'].replace('Land', 'Land Rover')

In [66]:
data.isna().mean()

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine           0.027190
max_power        0.026575
torque           0.027313
seats            0.027190
dtype: float64

In [67]:
data.dropna(inplace=True)
len(data)

7906

In [68]:
data = pd.get_dummies(data, columns=['name', 'fuel', 'seller_type', 'transmission', 'owner'], drop_first=True)

In [69]:
data.head()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0


#### Split the train and test dataset

In [70]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(data.drop('selling_price', axis=1), data['selling_price'], random_state=5, test_size=0.7)

In [71]:
print(len(X_train), len(X_test))

2371 5535


In [73]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=100)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [77]:
from sklearn.metrics import mean_squared_error
train_rmse = mean_squared_error(y_train, train_pred) ** 0.5
test_rmse = mean_squared_error(y_test, test_pred) ** 0.5
print(f'train_rmse: {train_rmse}, test_rmse: {test_rmse}')

train_rmse: 63717.21392327156, test_rmse: 189926.6645841533


In [78]:
from sklearn.model_selection import KFold

In [79]:
data

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,2013,320000,110000,0.230014,1197.0,82.85,113.70000,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
8124,2007,135000,119000,0.228385,1493.0,110.00,235.35840,5.0,0,0,...,1,0,0,1,0,1,1,0,0,0
8125,2009,382000,120000,0.262371,1248.0,73.90,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
8126,2013,290000,25000,0.320419,1396.0,70.00,140.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0


In [80]:
data.reset_index(drop=True, inplace=True) # 인덱스를 변수로 가져옴, drop=True해주지 않으면 새로운 컬럼 형태로 추가 되기 때문에 반드시 해준다.

In [81]:
data

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7901,2013,320000,110000,0.230014,1197.0,82.85,113.70000,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
7902,2007,135000,119000,0.228385,1493.0,110.00,235.35840,5.0,0,0,...,1,0,0,1,0,1,1,0,0,0
7903,2009,382000,120000,0.262371,1248.0,73.90,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
7904,2013,290000,25000,0.320419,1396.0,70.00,140.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0


In [82]:
kf = KFold(n_splits=5)

In [83]:
X = data.drop('selling_price', axis=1)
y = data['selling_price']

In [85]:
for i, j in kf.split(X): 
    print(i, j)
    
    # TrainSet, TestSet
#1
#2
#3
#4
#5

[1582 1583 1584 ... 7903 7904 7905] [   0    1    2 ... 1579 1580 1581]
[   0    1    2 ... 7903 7904 7905] [1582 1583 1584 ... 3160 3161 3162]
[   0    1    2 ... 7903 7904 7905] [3163 3164 3165 ... 4741 4742 4743]
[   0    1    2 ... 7903 7904 7905] [4744 4745 4746 ... 6322 6323 6324]
[   0    1    2 ... 6322 6323 6324] [6325 6326 6327 ... 7903 7904 7905]


In [87]:
train_rmse_total = []
test_rmse_total = []
for train_index, test_index in kf.split(X): 
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = RandomForestRegressor(random_state=100)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_rmse= mean_squared_error(y_train, train_pred) ** 0.5
    test_rmse = mean_squared_error(y_test, test_pred) ** 0.5
    
    train_rmse_total.append(train_rmse)
    test_rmse_total.append(test_rmse)
    
    

In [88]:
print ('train_rmse: ', sum(train_rmse_total)/ 5, 'test_rmse: ', sum(test_rmse_total)/ 5)

train_rmse:  56553.836119114814 test_rmse:  142936.58918244042


n_estimators: 결정 트리의 개수, 기본 100개 

max_depth: 각 트리의 최대 깊이를 제한 

min_samples_split: 해당 노드를 나눌 것인지 말 것인지를 노드 데이터 수를 기준으로 판단, 기본값: 2// 숫자가 클수록 오버피팅을 피한다.

min_samples_leaf: 분리된 노드에 최소 몇개의 데이터가 있어야 하는지, 기본값: 1 // 숫자가 클수록 오버피팅을 피한다 

n_jobs: 병렬 처리에 사용되는 cpu 코어수 : 기본값은 None

In [89]:
train_rmse_total = []
test_rmse_total = []
for train_index, test_index in kf.split(X): 
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = RandomForestRegressor(n_estimators=300, max_depth=50, min_samples_split=5, min_samples_leaf=1, n_jobs=1, random_state=100)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    train_rmse= mean_squared_error(y_train, train_pred) ** 0.5
    test_rmse = mean_squared_error(y_test, test_pred) ** 0.5
    
    train_rmse_total.append(train_rmse)
    test_rmse_total.append(test_rmse)
    
    

In [90]:
print ('train_rmse: ', sum(train_rmse_total)/ 5, 'test_rmse: ', sum(test_rmse_total)/ 5)

train_rmse:  66762.84568886801 test_rmse:  142205.83441414658
